In [11]:
# Importar librerías
import googlemaps
import streamlit as st
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from streamlit_folium import st_folium
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='streamlit')

In [3]:
# Cargar el modelo de predicción
modelo_ridge = joblib.load('./models/modelo_ridge.joblib')

# Cargar datos de prueba (pueden ser datos históricos de viajes en Nueva York)
# Aquí puedes usar tus propios datos o conjuntos de datos de ejemplo
datos_viajes = pd.read_parquet('./data/taxis_clean.parquet')
viajes = datos_viajes.iloc[1000000:3000000]

# Títulos y Descripción
st.title('Predicción de Precios de Viajes en Nueva York')
st.write('Aplicación para predecir precios de viajes en función de la distancia y el tiempo.')

2024-09-19 19:19:37.344 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:38.862 
  command:

    streamlit run c:\Users\Andres\Documents\ANDRES\Programacion\Proyectos\ML_proyect\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-19 19:19:38.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:38.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:38.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:38.868 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:38.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [7]:
viajes.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pulocationid,dolocationid,payment_type,fare_amount,total_amount,zone,borough,trip_duration,pickup_hour,distance_bin
1000000,VeriFone Inc,2024-01-01 18:57:45,2024-01-01 19:06:02,1.0,2.45,170,140,Credit card,12.1,19.32,Murray Hill,Manhattan,8.283333,18,2-5
1000001,VeriFone Inc,2024-01-01 18:57:45,2024-01-01 19:06:02,1.0,2.45,170,140,Credit card,12.1,19.32,Murray Hill,Manhattan,8.283333,18,2-5
1000002,VeriFone Inc,2024-01-01 18:57:45,2024-01-01 19:06:02,1.0,2.45,170,140,Credit card,12.1,19.32,Murray Hill,Manhattan,8.283333,18,2-5
1000003,VeriFone Inc,2024-01-01 18:57:45,2024-01-01 19:06:02,1.0,2.45,170,140,Credit card,12.1,19.32,Murray Hill,Manhattan,8.283333,18,2-5
1000004,VeriFone Inc,2024-01-01 18:57:45,2024-01-01 19:06:02,1.0,2.45,170,140,Credit card,12.1,19.32,Murray Hill,Manhattan,8.283333,18,2-5


In [4]:
# Obtener las zonas únicas para los selectbox
zonas_unicas = viajes['zone'].unique()

# Entrada de usuario para seleccionar origen y destino
st.write('### Seleccione el origen y el destino:')
zona_origen = st.selectbox('Seleccionar origen', zonas_unicas)
zona_destino = st.selectbox('Seleccionar destino', zonas_unicas)

# Calcular la distancia automáticamente usando el DataFrame
# Filtramos el DataFrame para obtener los viajes entre las zonas seleccionadas
distancia = viajes.loc[(viajes['zone'] == zona_origen) & (viajes['pulocationid'] == zona_destino), 'trip_distance']

# Si no hay un viaje registrado, manejar el caso
if distancia.empty:
    st.write("No se encontraron viajes entre estas zonas. Intenta con otras zonas.")
else:
    # Obtener la primera distancia (puedes usar estadísticas como promedio si es necesario)
    distancia = distancia.iloc[0]

    # Entrada de usuario: tiempo estimado
    tiempo_estimado = st.number_input('Ingrese el tiempo estimado del viaje (en minutos)', min_value=0)

    # Realizar predicción de precio
    if st.button('Predecir Precio'):
        # Ajustar los datos para el modelo
        input_data = np.array([[distancia, tiempo_estimado]])
        
        # Predecir usando el modelo cargado
        precio_predicho = modelo_ridge.predict(input_data)[0]
        
        st.write(f'El precio estimado del viaje de {zona_origen} a {zona_destino} es: ${precio_predicho:.2f}')

2024-09-19 19:19:39.148 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:19:39.168 Session state does not function when running a script without `streamlit run`
2024-09-19 19:19

## Calidad del aire

In [8]:
# Cargar los datos de calidad del aire
calidad_aire = pd.read_csv('./data/CD_air.csv')
calidad_aire.head()

,date,id_cd,cd_name,id_borough,borough_name,polluting_agent,data_value
0,2022-06-01,307.0,Sunset Park (CD7),2,brooklyn,Fine particles (PM 2.5),6.7
1,2022-06-01,313.0,Coney Island (CD13),2,brooklyn,O3 (ppb),37.7
2,2022-06-01,105.0,Midtown (CD5),1,manhattan,Fine particles (PM 2.5),8.7
3,2022-06-01,107.0,Upper West Side (CD7),1,manhattan,O3 (ppb),29.9
4,2022-06-01,315.0,Sheepshead Bay (CD15),2,brooklyn,Fine particles (PM 2.5),5.8


In [9]:
# Título y Descripción
st.title('Predicción de la Calidad del Aire en Nueva York')
st.write('Aplicación para visualizar y predecir la calidad del aire en diferentes zonas de la ciudad.')

# Obtener las zonas únicas
zonas_aire = calidad_aire['cd_name'].unique()

# Entrada de usuario: selección de la zona
st.write('### Seleccione la zona para ver la calidad del aire:')
zona_seleccionada = st.selectbox('Seleccionar zona', zonas_aire)

# Filtrar los datos por la zona seleccionada
datos_zona = calidad_aire[calidad_aire['cd_name'] == zona_seleccionada]

# Mostrar los datos de calidad del aire para esa zona
st.write(f'### Datos de calidad del aire para {zona_seleccionada}:')
st.dataframe(datos_zona[['polluting_agent', 'data_value']])

# Gráfico de calidad del aire en esa zona (opcional)
if st.button('Mostrar Gráfico de Contaminantes'):
    st.bar_chart(datos_zona.set_index('polluting_agent')['data_value'])

2024-09-19 19:54:56.005 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.006 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.009 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:54:56.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Podemos relacionar las zonas de los viajes con los datos de calidad del aire para que el usuario vea cómo afecta la calidad del aire en la zona de destino del viaje. Esto podría involucrar:

* Relacionar la zona del destino con los datos de calidad del aire.
* Mostrar la calidad del aire al predecir el precio del viaje o al seleccionar la zona de destino.

In [10]:
# Mostrar calidad del aire para la zona de destino seleccionada en los viajes
st.write(f'### Calidad del aire en la zona de destino ({zona_destino}):')

# Filtrar los datos de calidad del aire para la zona seleccionada
aire_destino = calidad_aire[calidad_aire['cd_name'] == zona_destino]

# Mostrar los contaminantes
st.dataframe(aire_destino[['polluting_agent', 'data_value']])

2024-09-19 19:56:14.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:56:14.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:56:14.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:56:14.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:56:14.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 19:56:14.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

## Predicción del CO2 según la distancia y tipo de vehículo

In [ ]:
# Entrada de usuario: Selección del tipo de vehículo
st.write('### Seleccione el tipo de vehículo:')
tipo_vehiculo = st.selectbox('Tipo de vehículo', ['Gasolina', 'Híbrido', 'Eléctrico'])

# Fórmula simple para estimar emisiones de CO2 (ejemplo)
if tipo_vehiculo == 'Gasolina':
    co2_emisiones = distancia * 0.21  # kg de CO2 por km
elif tipo_vehiculo == 'Híbrido':
    co2_emisiones = distancia * 0.12  # kg de CO2 por km
else:
    co2_emisiones = distancia * 0.05  # kg de CO2 por km (para eléctrico)

# Mostrar las emisiones de CO2 estimadas
st.write(f'Emisiones estimadas de CO2 para un vehículo {tipo_vehiculo}: {co2_emisiones:.2f} kg de CO2')


## Mapa

In [14]:
from dotenv import load_dotenv
import os
import folium
from opencage.geocoder import OpenCageGeocode

# Cargar las variables del archivo .env
load_dotenv()

# API key de OpenCage
api_key = os.getenv("OPENCAGE_API_KEY")
geocoder = OpenCageGeocode(api_key)

# Ejemplo de nombres de zonas
zonas = ['Murray Hill, Manhattan', 'Times Square, Manhattan']

# Crear un mapa centrado en Nueva York
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Geocodificar cada zona y añadir marcadores al mapa
for zona in zonas:
    result = geocoder.geocode(zona)
    if result:
        lat = result[0]['geometry']['lat']
        lon = result[0]['geometry']['lng']
        folium.Marker(
            location=[lat, lon],
            popup=zona
        ).add_to(m)

# Mostrar el mapa en Jupyter
m

In [12]:
import folium
from geopy.geocoders import Nominatim

# Iniciar Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

# Ejemplo de nombres de zonas
zonas = ['Murray Hill, Manhattan', 'Times Square, Manhattan']

# Crear un mapa centrado en Nueva York
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Geocodificar cada zona y añadir marcadores al mapa
for zona in zonas:
    location = geolocator.geocode(zona)
    if location:
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=zona
        ).add_to(m)

# Mostrar el mapa en Jupyter
m